# Test deployed web application

In [6]:
app_url = '52.186.126.119'

In [7]:
scoring_url = 'http://{}/score'.format(app_url)
version_url = 'http://{}/version'.format(app_url)

In [8]:
version_url

'http://52.186.126.119/version'

In [9]:
!curl $version_url # Reports the tensorflow version

^C
